# Hooks를 사용한 Strands Agent와 AgentCore Memory 튜토리얼

## 개요

이 튜토리얼은 Hooks를 통해 AgentCore Memory와 통합된 Strands agent를 사용하여 지능형 개인 어시스턴트를 구축하는 방법을 보여줍니다. Agent는 대화 컨텍스트를 유지하고 상호작용을 통해 학습하여 개인화된 응답을 제공합니다.

## 튜토리얼 세부사항

**사용 사례**: 수학 어시스턴트

| 정보                | 세부사항                                                                          |
|:--------------------|:---------------------------------------------------------------------------------|
| 튜토리얼 유형        | 장기 대화형                                                                       |
| Agent 유형          | 수학 어시스턴트                                                                   |
| Agentic Framework   | Strands Agents                                                                   |
| LLM model           | Anthropic Claude Haiku 4.5                                                      |
| 튜토리얼 구성요소    | Memory를 위한 AgentCore Summary Strategy, Memory 저장 및 검색을 위한 Hooks       |
| 예제 복잡도          | 중급                                                                             |


학습 내용:
- 대화 요약을 사용한 AgentCore Memory 설정
- 자동 저장 및 검색을 위한 memory hooks 생성
- 지속적인 메모리를 가진 Strands agent 구축
- 대화 전반에 걸친 메모리 기능 테스트

### 시나리오 컨텍스트

이 예제에서는 이전 대화의 요약을 저장하는 수학 어시스턴트 예제를 만듭니다.
이 예제의 주요 기능:
- **자동 메모리 저장**: 대화가 자동으로 저장됩니다
- **컨텍스트 검색**: 이전 대화가 현재 응답에 반영됩니다
- **요약 생성**: 주요 정보가 추출되고 요약됩니다
- **Tool 통합**: 수학 연산을 위한 계산기 도구

## 아키텍처
<div style="text-align:left">
    <img src="architecture.png" width="65%" />
</div>


## 사전 요구사항

이 튜토리얼을 실행하려면 다음이 필요합니다:
- Python 3.10+
- Amazon Bedrock AgentCore Memory 권한이 있는 AWS 자격 증명
- Amazon Bedrock AgentCore SDK

## Step 1: 환경 설정
이 노트북을 작동시키기 위해 필요한 모든 라이브러리를 가져오고 클라이언트를 정의하는 것부터 시작하겠습니다.

In [ ]:
!pip install -qr requirements.txt

In [ ]:
# AgentCore Memory 클라이언트와 전략 타입 import
from bedrock_agentcore.memory import MemoryClient
from bedrock_agentcore.memory.constants import StrategyType

In [ ]:
import os
import logging
from strands import Agent  # Strands Agent framework
from datetime import datetime
from strands_tools import calculator  # 수학 계산 tool
from strands.hooks import AfterInvocationEvent, HookProvider, HookRegistry, MessageAddedEvent  # Hook 시스템

# Setup logging
logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")
logger = logging.getLogger("memory-tutorial")

# Configuration - replace with your values
REGION = os.getenv('AWS_REGION', 'us-west-2')
ROLE_ARN = "<<INSERT-YOUR-IAM-ROLE>>"  # AgentCore Memory 권한이 있는 IAM Role ARN
ACTOR_ID = f"actor-{datetime.now().strftime('%Y%m%d%H%M%S')}"  # 사용자 식별자
SESSION_ID = f"tutorial-{datetime.now().strftime('%Y%m%d%H%M%S')}"  # 세션 식별자

## Step 2: Memory 리소스 생성

이 단계에서는 요약 전략을 사용하여 memory 리소스를 생성합니다. 이 리소스는 대화 데이터를 저장하고 구성합니다. 정의하는 전략은 대화 요약을 자동으로 생성하고 구성된 네임스페이스에 저장합니다.

먼저, 수학 어시스턴트를 위한 커스텀 프롬프트를 생성하겠습니다.


In [ ]:
CUSTOM_PROMPT = """
Your task is to extract math learning data from the user's conversations. You store the progress of the user in a memory system to understand their math level and help them progress.

You are tasked with analyzing conversations to extract the user's math learning patterns. You'll be analyzing two sets of data: 

<past_conversation> 
[Past conversations between the user and math tutor will be placed here for context] 
</past_conversation> 

<current_conversation> 
[The current conversation between the user and math tutor will be placed here] 
</current_conversation> 

Your job is to identify and categorize the user's math learning profile:
- Extract the user's current math level from problems they solve correctly/incorrectly
- Extract the user's preferred learning style from how they ask questions and respond to explanations
- Extract topic strengths and weaknesses from their performance patterns
- Track learning progress and identify areas needing reinforcement
"""

In [ ]:
from botocore.exceptions import ClientError

# Initialize Memory Client
client = MemoryClient(region_name=REGION)
memory_name = "MathAssistant"
# Define memory strategy for conversation summaries
strategies = [
    {
        StrategyType.CUSTOM.value: {
            "name": "CustomSemanticMemory",
            "description": "Captures facts from conversations",
            "namespaces": ["/students/math/{actorId}/"],  # actorId별로 메모리 저장 경로 구성
            "configuration" : {
                "semanticOverride" : {
                    "extraction" : {
                        "modelId" : "global.anthropic.claude-haiku-4-5-20251001-v1:0",
                        "appendToPrompt": CUSTOM_PROMPT  # 커스텀 프롬프트로 학습 패턴 추출
                    }
                },
    }}}
]

# Create memory resource
try:
    memory = client.create_memory_and_wait(
        name=memory_name,
        strategies=strategies, # Use the defined long term strategies
        description="Memory for tutorial agent",
        event_expiry_days=30,  # 30일 후 이벤트 만료
        memory_execution_role_arn=ROLE_ARN,
    )
    memory_id = memory['id']
    logger.info(f"✅ Created memory: {memory_id}")
except ClientError as e:
    if e.response['Error']['Code'] == 'ValidationException' and "already exists" in str(e):
        # If memory already exists, retrieve its ID
        memories = client.list_memories()
        memory_id = next((m['id'] for m in memories if m['id'].startswith(memory_name)), None)
        logger.info(f"Memory already exists. Using existing memory ID: {memory_id}")
except Exception as e:
    # Handle any errors during memory creation
    logger.info(f"❌ ERROR: {e}")
    import traceback
    traceback.print_exc()
    # Cleanup on error - delete the memory if it was partially created
    if memory_id:
        try:
            client.delete_memory_and_wait(memory_id=memory_id)
            logger.info(f"Cleaned up memory: {memory_id}")
        except Exception as cleanup_error:
            logger.info(f"Failed to clean up memory: {cleanup_error}")

## Step 3: Memory Hook Provider 생성

이 단계에서는 메모리 작업을 자동화하는 커스텀 `MemoryHookProvider` 클래스를 정의합니다. Hooks는 agent의 실행 라이프사이클의 특정 시점에서 실행되는 특수 함수입니다. 생성하는 memory hook은 두 가지 주요 기능을 제공합니다:

1. **메모리 검색**: 사용자가 메시지를 보낼 때 관련된 과거 대화를 자동으로 가져옵니다
2. **메모리 저장**: agent가 응답한 후 새로운 대화를 저장합니다

이를 통해 수동 관리 없이 원활한 메모리 경험을 제공합니다.

In [ ]:
class MemoryHookProvider(HookProvider):
    """Hook provider for automatic memory management"""
    
    def __init__(self, memory_id: str, client: MemoryClient):
        self.memory_id = memory_id
        self.client = client
    
    def retrieve_memories(self, event: MessageAddedEvent):
        """Retrieve relevant memories before processing user message"""
        messages = event.agent.messages
        # 마지막 메시지가 user 메시지이고 tool 결과가 아닌 경우에만 메모리 검색
        if messages[-1]["role"] == "user" and "toolResult" not in messages[-1]["content"][0]:
            user_message = messages[-1]["content"][0].get("text", "")
            
            try:
                # Get actor_id from agent state
                actor_id = event.agent.state.get("actor_id")
                if not actor_id:
                    logger.warning("Missing actor_id in agent state")
                    return
                
                namespace = f"/students/math/{actor_id}/"
                
                # Retrieve relevant memories
                memories = self.client.retrieve_memories(
                    memory_id=self.memory_id,
                    namespace=namespace,
                    query=user_message  # 사용자 메시지와 관련된 메모리 검색
                )
                
                # Extract memory content
                memory_context = []
                for memory in memories:
                    if isinstance(memory, dict):
                        content = memory.get('content', {})
                        if isinstance(content, dict):
                            text = content.get('text', '').strip()
                            if text:
                                memory_context.append(text)
                
                # Inject memories into user message
                if memory_context:
                    context_text = "\n".join(memory_context)
                    original_text = messages[-1]["content"][0].get("text", "")
                    # 원본 메시지에 과거 컨텍스트를 추가하여 agent에게 전달
                    messages[-1]["content"][0]["text"] = (
                        f"{original_text}\n\nPrevious context: {context_text}"
                    )
                    logger.info(f"Retrieved {len(memory_context)} memories")
                    
            except Exception as e:
                logger.error(f"Failed to retrieve memories: {e}")
    
    def save_memories(self, event: AfterInvocationEvent):
        """Save conversation after agent response"""
        try:
            messages = event.agent.messages
            if len(messages) >= 2 and messages[-1]["role"] == "assistant":
                # Get last user and assistant messages
                user_msg = None
                assistant_msg = None
                
                # 역순으로 순회하여 가장 최근의 user-assistant 메시지 쌍 추출
                for msg in reversed(messages):
                    if msg["role"] == "assistant" and not assistant_msg:
                        assistant_msg = msg["content"][0]["text"]
                    elif msg["role"] == "user" and not user_msg and "toolResult" not in msg["content"][0]:
                        user_msg = msg["content"][0]["text"]
                        break
                
                if user_msg and assistant_msg:
                    # Get session info from agent state
                    actor_id = event.agent.state.get("actor_id")
                    session_id = event.agent.state.get("session_id")
                    
                    if not actor_id or not session_id:
                        logger.warning("Missing actor_id or session_id in agent state")
                        return
                    
                    # Save conversation
                    self.client.create_event(
                        memory_id=self.memory_id,
                        actor_id=actor_id,
                        session_id=session_id,
                        messages=[(user_msg, "USER"), (assistant_msg, "ASSISTANT")]  # 대화 쌍을 이벤트로 저장
                    )
                    logger.info("Saved conversation to memory")
                    
        except Exception as e:
            logger.error(f"Failed to save memories: {e}")
    
    def register_hooks(self, registry: HookRegistry) -> None:
        """Register memory hooks"""
        # MessageAddedEvent: 메시지 추가 시 메모리 검색
        registry.add_callback(MessageAddedEvent, self.retrieve_memories)
        # AfterInvocationEvent: agent 응답 후 메모리 저장
        registry.add_callback(AfterInvocationEvent, self.save_memories)
        logger.info("Memory hooks registered")

## Step 4: Memory를 가진 Agent 생성

이제 Strands agent를 생성하고 memory hook provider와 연결합니다. 이 agent는 두 가지 주요 기능을 갖습니다:

1. **Memory 통합**: 생성한 memory hooks가 자동 컨텍스트 검색을 가능하게 합니다
2. **계산기 Tool**: Agent가 필요할 때 수학 연산을 수행할 수 있습니다

이 조합은 과거 상호작용을 기억하고 유용한 계산을 수행할 수 있는 개인 어시스턴트를 만듭니다.

In [ ]:
# Create memory hook provider
memory_hooks = MemoryHookProvider(memory_id, client)

# Create agent with memory hooks and calculator tool
agent = Agent(
    hooks=[memory_hooks],  # 메모리 자동 저장/검색을 위한 hook 등록
    model = "global.anthropic.claude-haiku-4-5-20251001-v1:0",
    tools=[calculator],  # 수학 계산 tool
    system_prompt="You are a helpful personal math tutor. You assist users in solving math problems and provide personalized assistance.",
    state={"actor_id": ACTOR_ID, "session_id": SESSION_ID}  # 메모리 저장에 필요한 식별자
)

print("✅ Agent created with memory hooks.")

**Agent 설정이 완료되었습니다! 이제 테스트해보겠습니다.**

## Memory 기능 테스트

이 섹션에서는 일련의 상호작용을 통해 agent의 메모리 기능을 테스트합니다. Agent가 시간이 지남에 따라 컨텍스트를 구축하고 이전 상호작용을 기억하는 방법을 관찰합니다.

먼저, agent에게 자신을 소개하고 수학 질문을 해보겠습니다:

In [ ]:
# First interaction - introduce yourself
response1 = agent("안녕하세요!, 내 이름은 철수이고 방금 이산수학 과목에 등록했습니다. 이 문제를 풀도록 도와주세요: 책 5권을 책장에 배열할 수 있는 방법은 몇 가지인가요?")
print(f"Agent: {response1}")

Agent에게 다른 계산 작업을 주겠습니다:

In [ ]:
# Second interaction - another calculation
response2 = agent("저는 예제 문제와 함께 단계별 설명으로 배울 때 더 잘 이해합니다. 모듈러 연산(modular arithmetic)을 설명해 주실 수 있나요? 17 mod 5는 무엇인가요?")
print(f"Agent: {response2}")

이제 agent가 우리가 누구인지 기억하는지 확인해보겠습니다.

**참고:** 메모리가 추출, 통합 및 저장될 시간을 주기 위해 여기서 약 20초 정도 기다려주세요.

In [ ]:
# Third interaction - test memory recall
response3 = agent("모듈러 연산 다음에 공부해야 할 바로 다음 단계는 무엇인가요?")
print(f"Agent: {response3}")

마지막으로, agent가 우리의 계산 기록을 기억하는지 확인해보겠습니다:

In [ ]:
# Fourth interaction - test context awareness
response4 = agent("이건 너무 어려워요, 좀 더 쉬운건 없나요?")
print(f"Agent: {response4}")

### Memory 저장 확인

마지막 단계로, 대화가 AgentCore Memory에 제대로 저장되었는지 확인합니다. 이는 memory hooks가 올바르게 작동하고 있으며 agent가 향후 상호작용에서 이 정보에 액세스할 수 있음을 보여줍니다.

In [ ]:
# Check stored memories
try:
    memories = client.retrieve_memories(
        memory_id=memory_id,
        namespace=f"/students/math/{ACTOR_ID}/",
        query="mathematics calculations"  # 수학 관련 메모리 검색
    )
    
    print(f"\n📚 Found {len(memories)} memories:")
    for i, memory in enumerate(memories, 1):
        if isinstance(memory, dict):
            content = memory.get('content', {})
            if isinstance(content, dict):
                text = content.get('text', '')[:200] + "..."
                print(f"{i}. {text}")
                
except Exception as e:
    print(f"Error retrieving memories: {e}")

튜토리얼 완료! 🎉

주요 요점:
- Memory hooks는 대화 컨텍스트를 자동으로 저장하고 검색합니다
- Agent는 여러 상호작용에 걸쳐 상태를 유지할 수 있습니다
- AgentCore Memory는 관련 컨텍스트에 대한 의미론적 검색을 제공합니다
- Tool은 메모리와 결합하여 향상된 기능을 제공할 수 있습니다

## 정리

### 선택사항: Memory 리소스 삭제

튜토리얼을 완료한 후, 불필요한 비용이 발생하지 않도록 memory 리소스를 삭제할 수 있습니다. 다음 코드는 정리를 위해 제공되지만 기본적으로 주석 처리되어 있습니다.

In [ ]:
# Uncomment to delete the memory resource
# try:
#     client.delete_memory_and_wait(memory_id=memory_id)
#     print(f"✅ Deleted memory resource: {memory_id}")
# except Exception as e:
#     print(f"Error deleting memory: {e}")